In [1]:
from utils.data_scrap_functions import *

Ce notebook est temporaire, il sera remplacé par un moyen plus simple et ergonomique d'importer les données.

## Importation des données

### Dans un premier temps, on importe la liste de toute les variables à récupérer

In [2]:
url = 'https://classic.clinicaltrials.gov/api/info/study_fields_list'

In [3]:
list_fields = get_list(url)

In [4]:
list_fields[:20]

['Acronym',
 'AgreementOtherDetails',
 'AgreementPISponsorEmployee',
 'AgreementRestrictionType',
 'AgreementRestrictiveAgreement',
 'ArmGroupDescription',
 'ArmGroupInterventionName',
 'ArmGroupLabel',
 'ArmGroupType',
 'AvailIPDComment',
 'AvailIPDId',
 'AvailIPDType',
 'AvailIPDURL',
 'BaselineCategoryTitle',
 'BaselineClassDenomCountGroupId',
 'BaselineClassDenomCountValue',
 'BaselineClassDenomUnits',
 'BaselineClassTitle',
 'BaselineDenomCountGroupId',
 'BaselineDenomCountValue']

### Maintenant, on importe les données depuis l'API de Clinical Trials selon la liste de variable qu'on veut

In [5]:
get_data(taille=10,list_fields=list_fields[-40:])

100%|██████████| 10/10 [01:01<00:00,  6.16s/it]
/home/onyxia/work/ProjetPython/utils/data_scrap_functions.py:95: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  full_df = full_df.applymap(treat_list)


Temps requete API: -------------------- 61.31017231941223 

 Temps Pandas: -------------------- 0.27068138122558594


,RetractionSource,SamplingMethod,SecondaryId,SecondaryIdDomain,SecondaryIdLink,SecondaryIdType,SecondaryOutcomeDescription,SecondaryOutcomeMeasure,SecondaryOutcomeTimeFrame,SeeAlsoLinkLabel,...,SubmissionMCPReleaseN,SubmissionReleaseDate,SubmissionResetDate,SubmissionUnreleaseDate,TargetDuration,UnpostedEventDate,UnpostedEventType,UnpostedResponsibleParty,VersionHolder,WhyStopped
0,None,None,None,None,None,None,Number of opioid pill used if opioid prescript...,Opioid pills used,25-40 days after surgery,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
1,None,None,1R01AG082892-01,None,https://reporter.nih.gov/quickSearch/1R01AG082...,U.S. NIH Grant/Contract,The perceived stress scale has 10 items rated ...,Perceived Stress Scale,"Collected at: enrollment, 3- and 6-months afte...",None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
3,None,Non-Probability Sample,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
4,None,None,None,None,None,None,Adverse Events (AEs) and Serious Adverse Event...,Adverse Events,Day 0 through Day 5 visit,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,None,None,None,None,None,None,Overall survival,overall survival (OS),2 years,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
9996,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
9997,None,Non-Probability Sample,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
9998,None,None,None,None,None,None,8 multiple choice questions (5 items per MCQ);...,knowledge of student,15 days after simulation session,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None


In [6]:
df = get_data_parralel(taille=100,list_fields=list_fields)
df

 33%|███▎      | 33/100 [03:05<05:47,  5.19s/it]

Erreur de connexion à l'url https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=InterventionBrowseLeafName%2CInterventionBrowseLeafRelevance%2CInterventionDescription%2CInterventionMeshId%2CInterventionMeshTerm%2CInterventionName%2CInterventionOtherName%2CInterventionType%2CIsFDARegulatedDevice%2CIsFDARegulatedDrug%2CIsPPSD%2CIsUSExport%2CIsUnapprovedDevice%2CKeyword%2CLargeDocDate%2CLargeDocFilename%2CLargeDocHasICF%2CLargeDocHasProtocol%2CLargeDocHasSAP%2CLargeDocLabel&min_rnk=33001&max_rnk=34000&fmt=json: 503 Server Error: Service Unavailable for url: https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=InterventionBrowseLeafName%2CInterventionBrowseLeafRelevance%2CInterventionDescription%2CInterventionMeshId%2CInterventionMeshTerm%2CInterventionName%2CInterventionOtherName%2CInterventionType%2CIsFDARegulatedDevice%2CIsFDARegulatedDrug%2CIsPPSD%2CIsUSExport%2CIsUnapprovedDevice%2CKeyword%2CLargeDocDate%2CLargeDocFilename%2CLargeDocHasICF%2CLargeD

 34%|███▍      | 34/100 [03:31<12:33, 11.42s/it]

Erreur de connexion à l'url https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=SeriousEventNotes%2CSeriousEventOrganSystem%2CSeriousEventSourceVocabulary%2CSeriousEventStatsGroupId%2CSeriousEventStatsNumAffected%2CSeriousEventStatsNumAtRisk%2CSeriousEventStatsNumEvents%2CSeriousEventTerm%2CStartDate%2CStartDateType%2CStatusVerifiedDate%2CStdAge%2CStudyFirstPostDate%2CStudyFirstPostDateType%2CStudyFirstSubmitDate%2CStudyFirstSubmitQCDate%2CStudyPopulation%2CStudyType%2CSubmissionMCPReleaseN%2CSubmissionReleaseDate&min_rnk=34001&max_rnk=35000&fmt=json: 500 Server Error: Internal Server Error for url: https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=SeriousEventNotes%2CSeriousEventOrganSystem%2CSeriousEventSourceVocabulary%2CSeriousEventStatsGroupId%2CSeriousEventStatsNumAffected%2CSeriousEventStatsNumAtRisk%2CSeriousEventStatsNumEvents%2CSeriousEventTerm%2CStartDate%2CStartDateType%2CStatusVerifiedDate%2CStdAge%2CStudyFirstPostDate%2CStudyFirstPo

 35%|███▌      | 35/100 [03:58<17:16, 15.95s/it]

...

 96%|█████████▌| 96/100 [14:13<00:43, 10.77s/it]

Erreur de connexion à l'url https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=ExpandedAccessNCTId%2CExpandedAccessStatusForNCTId%2CFDAAA801Violation%2CFlowAchievementComment%2CFlowAchievementGroupId%2CFlowAchievementNumSubjects%2CFlowAchievementNumUnits%2CFlowDropWithdrawComment%2CFlowDropWithdrawType%2CFlowGroupDescription%2CFlowGroupId%2CFlowGroupTitle%2CFlowMilestoneComment%2CFlowMilestoneType%2CFlowPeriodTitle%2CFlowPreAssignmentDetails%2CFlowReasonComment%2CFlowReasonGroupId%2CFlowReasonNumSubjects%2CFlowReasonNumUnits&min_rnk=96001&max_rnk=97000&fmt=json: 503 Server Error: Service Unavailable for url: https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=ExpandedAccessNCTId%2CExpandedAccessStatusForNCTId%2CFDAAA801Violation%2CFlowAchievementComment%2CFlowAchievementGroupId%2CFlowAchievementNumSubjects%2CFlowAchievementNumUnits%2CFlowDropWithdrawComment%2CFlowDropWithdrawType%2CFlowGroupDescription%2CFlowGroupId%2CFlowGroupTitle%2CFlowMileston

100%|██████████| 100/100 [15:14<00:00,  9.15s/it]
/home/onyxia/work/ProjetPython/utils/data_scrap_functions.py:155: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  full_df = full_df.applymap(treat_list)


Temps requete API: -------------------- 827.6035621166229 

 Temps Pandas: -------------------- 86.72732901573181


,Acronym,AgreementOtherDetails,AgreementPISponsorEmployee,AgreementRestrictionType,AgreementRestrictiveAgreement,ArmGroupDescription,ArmGroupInterventionName,ArmGroupLabel,ArmGroupType,AvailIPDComment,...,SubmissionMCPReleaseN,SubmissionReleaseDate,SubmissionResetDate,SubmissionUnreleaseDate,TargetDuration,UnpostedEventDate,UnpostedEventType,UnpostedResponsibleParty,VersionHolder,WhyStopped
0,None,None,None,None,None,Patients in this group will be administered a ...,Other: Opioid Use Agreement,Opioid Use Agreement,Experimental,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
1,iCare4Me T,None,None,None,None,Digital health information is delivered throug...,Other: Digital Health Information,Control group,Active Comparator,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
2,None,None,None,None,None,None,Device: VERTICA Active device,Active,Experimental,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
3,None,None,None,None,None,A retrospective cohort of cancer patients base...,None,Cancer patients cohort in SEER database,None,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
4,None,None,None,None,None,Period 1: SC Period 2: IV,Drug: SCP-111,Treatment Sequence 1,Experimental,None,...,None,None,None,None,None,None,None,None,"December 12, 2023",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,None,None,None,None,None,We applied 0.1% silane fluoride (Fluor Protect...,Drug: Pregnant women to whom applied silane fl...,Pregnant women to whom applied silane fluoride,Experimental,None,...,None,None,None,None,None,None,None,None,"December 13, 2023",None
99996,None,MedImmune has 60 days to review results commun...,No,OTHER,Yes,Participants will use the AMAZE™ application t...,Device: AMAZE™ Application,Massachusetts General Hospital (MGH) Participants,Experimental,None,...,None,None,None,None,None,None,None,None,"December 13, 2023",None
99997,HL,None,None,None,None,The intervention group received four weekly tr...,Other: Health Literacy Education,Intervention,Experimental,None,...,None,None,None,None,None,None,None,None,"December 13, 2023",None
99998,FIRST,None,None,None,None,A refresher training course will be the equiva...,Behavioral: Refresher course,Usual care training,Active Comparator,None,...,None,None,None,None,None,None,None,None,"December 13, 2023",None


In [7]:
df.to_csv('./data/clini_data.csv')